In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
df = pd.read_html('http://www.upsiapp.com/games/578081', attrs = {'class': 'scores'})
df = df[0]

In [3]:
a = ['Player', 'Score']
a.extend([str(i) for i in range(1,len(df.columns)-1)])

df.columns = a
df

,Player,Score,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,Sami T.,56,2,4,3,3,3,3,3,3,...,2,2,3,3,3,3,3,2,4,2
1,Hemmo S.,58,3,3,3,3,3,3,2,3,...,4,3,3,3,3,3,3,3,2,3
2,Daniel T.,65,2,3,3,3,5,3,3,4,...,3,5,3,4,3,3,2,4,4,3


In [4]:
rows = []
for index, row in df.iterrows():
    to_input = []
    for obj in row:
        to_input.append(obj)
    rows.append(to_input)


In [21]:
rows

[['Sami T.', 56, 2, 4, 3, 3, 3, 3, 3, 3, 2, 3, 2, 2, 3, 3, 3, 3, 3, 2, 4, 2],
 ['Hemmo S.', 58, 3, 3, 3, 3, 3, 3, 2, 3, 3, 2, 4, 3, 3, 3, 3, 3, 3, 3, 2, 3],
 ['Daniel T.', 65, 2, 3, 3, 3, 5, 3, 3, 4, 3, 2, 3, 5, 3, 4, 3, 3, 2, 4, 4, 3]]

In [30]:
dg_db = sqlite3.connect('discgolf.db')
c = dg_db.cursor()

In [31]:

date = "2020-05-13"

In [32]:
for row in rows:
    name = row[0]
    c.execute("""SELECT playerID FROM player WHERE playerName = ?;""", (name,))
    p_id = c.fetchall()[0][0]
    date = "2020-05-13"

    c.execute('''
        INSERT INTO player_round ('playerID','date') VALUES(?,?);
        ''', (p_id, date))
    
    _id = c.lastrowid

    dg_db.commit()
    holes = [i for i in range(1,len(df.columns)-1)]
    
    for i in range(2,len(row)):
        c.execute('''
            INSERT INTO player_scores ('courseID','strokes','holeID', 'playerID', 'playerRoundID') VALUES(?,?,?,?,?)
            ''', (1,row[i], holes[i-2], p_id, _id))
        dg_db.commit()

        

In [33]:
dg_db.close()

0


3